**Movie Recommendation System Using SVD**

### Importing the basic libraries

In [ ]:
import numpy as np
import pandas as pd

### Importing & Parsing the dataset as ratings and movies details

In [ ]:

# Reading the ratings data
ratingData = pd.read_csv('ratings.dat',
    names=['user_id', 'movie_id', 'rating', 'timestamp'],
    engine='python', delimiter='::', header=None, encoding='latin1')

# Reading the movies data
movieData = pd.read_csv('movies.dat',
    names=['movie_id', 'title', 'genre'],
    engine='python', delimiter='::', header=None, encoding='latin1')

# Printing the DataFrames
print(ratingData)
print(movieData)



         user_id  movie_id  rating  timestamp
0              1      1193       5  978300760
1              1       661       3  978302109
2              1       914       3  978301968
3              1      3408       4  978300275
4              1      2355       5  978824291
...          ...       ...     ...        ...
1000204     6040      1091       1  956716541
1000205     6040      1094       5  956704887
1000206     6040       562       5  956704746
1000207     6040      1096       4  956715648
1000208     6040      1097       4  956715569

[1000209 rows x 4 columns]
      movie_id                               title  \
0            1                    Toy Story (1995)   
1            2                      Jumanji (1995)   
2            3             Grumpier Old Men (1995)   
3            4            Waiting to Exhale (1995)   
4            5  Father of the Bride Part II (1995)   
...        ...                                 ...   
3878      3948             Meet the Parent

### Create the ratings matrix of shape (m×u)

In [ ]:
ratingMatrix = np.ndarray(
    shape=(np.max(ratingData.movie_id.values), np.max(ratingData.user_id.values)),
    dtype=np.uint8)
ratingMatrix[ratingData.movie_id.values-1, ratingData.user_id.values-1] = ratingData.rating.values
print(ratingMatrix)

[[  5  84  95 ...   0   0   3]
 [161   4   0 ...   0   0   0]
 [203   0   0 ... 127   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]


### Subtract Mean off - Normalization

In [ ]:
normalizedMatrix = ratingMatrix - np.asarray([(np.mean(ratingMatrix, 1))]).T
print(normalizedMatrix)

[[-1.80720199e+01  6.09279801e+01  7.19279801e+01 ... -2.30720199e+01
  -2.30720199e+01 -2.00720199e+01]
 [ 1.26374503e+02 -3.06254967e+01 -3.46254967e+01 ... -3.46254967e+01
  -3.46254967e+01 -3.46254967e+01]
 [ 1.72216887e+02 -3.07831126e+01 -3.07831126e+01 ...  9.62168874e+01
  -3.07831126e+01 -3.07831126e+01]
 ...
 [-3.27814570e-02 -3.27814570e-02 -3.27814570e-02 ... -3.27814570e-02
  -3.27814570e-02 -3.27814570e-02]
 [-2.58278146e-02 -2.58278146e-02 -2.58278146e-02 ... -2.58278146e-02
  -2.58278146e-02 -2.58278146e-02]
 [-2.42880795e-01 -2.42880795e-01 -2.42880795e-01 ... -2.42880795e-01
  -2.42880795e-01 -2.42880795e-01]]


### Computing SVD

In [ ]:
A = normalizedMatrix.T / np.sqrt(ratingMatrix.shape[0] - 1)
U, S, V = np.linalg.svd(A)

### Calculate cosine similarity, sort by most similar and return the top N

In [ ]:
def similar(ratingData, movie_id, top_n):
    index = movie_id - 1 # Movie id starts from 1
    movie_row = ratingData[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', ratingData, ratingData)) #Einstein summation |  traditional matrix multiplication and is equivalent to np.matmul(a,b)
    similarity = np.dot(movie_row, ratingData.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity) #Perform an indirect sort along the given axis (Last axis)
    return sort_indexes[:top_n]

### Select k principal components to represent the movies, a movie_id to find recommendations and print the top_n results

In [ ]:
k = 50
movie_id = 2
top_n = 5

sliced = V.T[:, :k] # representative data
indexes = similar(sliced, movie_id, top_n)

print('Recommendations for Movie {0}: \n'.format(
movieData[movieData.movie_id == movie_id].title.values[0]))
for id in indexes + 1:
    print(movieData[movieData.movie_id == id].title.values[0])

Recommendations for Movie Jumanji (1995): 

Jumanji (1995)
Santa Claus: The Movie (1985)
Antz (1998)
Honey, I Blew Up the Kid (1992)
All Dogs Go to Heaven (1989)


<ipython-input-9-a9dc92513cbb>:5: RuntimeWarning: invalid value encountered in divide
  similarity = np.dot(movie_row, ratingData.T) / (magnitude[index] * magnitude)
